<img src="Images/Logo.png" alt="Logo NSI" style="float:right">

<h1 style="text-align:center">TP : Traitement des images</h1>

# Description
Le projet consiste en la réalisation d’un petit logiciel de traitement d’images, permettant d’appliquer différents types de filtre, et possédant une interface 
graphique.
Cette interface sera réalisée avec Tkinter et contiendra trois zones :
* une image de départ, sur la gauche ;
* une série de bouton, au centre ;
* une image résultat, sur la droite.

Les différentes fonctionnalités à implémenter sont :
* charger une image sur laquelle travailler (elle s’affichera à gauche) ;
* choisir un filtre à appliquer à l’image de gauche en cliquant sur un bouton, 
l’image résultat s’affichera sur la partie droite ;

## Cahier des charges
Le contenu et les possibilités sont déjà cadrées dans le résumé du projet. Toutefois, si on souhaite s’écarter de cette description, il convient, en amont, de bien 
lister les fonctionnalités attendues.

La liste de ces fonctionnalités donnera des éléments utiles à la conception :
* combien d’images stockées en mémoire (ici deux, celle de gauche et celle de droite) ;
* disposition de l’interface (des boutons au centre sont convenables si on a une dizaine de transformation, mais il faudrait peut-être des menus s’il y en a plus).

## L’écriture d’un squelette
Les fonctionnalités complexes et un peu techniques sont : 
* la gestion de l’affichage ;
* le stockage des images. Il y a l’image affichée, l’image en mémoire, l’image de travail… 

Il est fondamental d’écrire un premier jet soigné contenant l’affichage des deux images et un bouton qui déclenche une action.

## Ajout des fonctionnalités
L’ajout des fonctionnalités est très simple si la base de travail est saine :
* ajout d’un bouton ;
* ajout d’une procédure qui implémente la transformation (de l’image A vers l’image B) ;
* connexion du bouton à la procédure, et rafraîchissement de l’affichage.

### Mise en place de la fenêtre principale et des widgets.
On utilise différents widgets pour notre projet :
* `Canvas` permet de disposer d'une fenêtre rectangulaire pour accueillir les images.
* `Frame` permet de regrouper et positionner d'autre widgets.Cahier des charges
* `Button` auquels on associera des actions 

In [ ]:
import tkinter as tk

fenetre = tk.Tk()
fenetre.title("Traitement images")

canevas_gauche = tk.Canvas(fenetre, width = 256, height = 400, background = "grey") # Ce composant permet d'afficher la photo originale
canevas_gauche.pack(padx = 8, pady = 8, side = tk.LEFT)     # On positionne les widgets les uns à côté des autres

panneau_boutons = tk.Frame(fenetre)    # Ce composant permet de postionner les boutons du projet
panneau_boutons.pack(side = tk.LEFT)

canevas_gauche = tk.Canvas(fenetre, width = 256, height = 400, background = "grey") # Ce composant permet d'afficher la photo après traitement
canevas_gauche.pack(padx = 8, pady = 8, side = tk.LEFT)

btn = tk.Button(panneau_boutons, text = "Ouvrir") # Ce bouton permet de selectionner l'image originale
btn.pack(fill = tk.X)

fenetre.mainloop()

### Chargement de l'image originale
On utilise la fonction `filedialog` du module `tkinter` qui permet de récupérer l'adresse d'un fichier sur l'ordinateur (pour après pouvoir l'ouvrir.

On utilise le module [`PIL`](https://pillow.readthedocs.io/en/stable/) qui permet de charger et de sauvegarder à peu près tous les formats d’image.

PIL permet de convertir des images au format Tkinter, qu’on pourra ensuite afficher : [ImageTk.PhotoImage()](https://pillow.readthedocs.io/en/stable/reference/ImageTk.html#PIL.ImageTk.PhotoImage)

Remarque : le module à installer s'appelle, en fait, [`pillow`](https://pillow.readthedocs.io/en/stable/).

In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import ImageTk, Image

fenetre = tk.Tk()
fenetre.title("Traitement images")

canevas_gauche = tk.Canvas(fenetre, width = 256, height = 400, background = "grey")
canevas_gauche.pack(padx = 8, pady = 8, side = tk.LEFT)

panneau_boutons = tk.Frame(fenetre)
panneau_boutons.pack(side = tk.LEFT)

canevas_droite = tk.Canvas(fenetre, width = 256, height = 400, background = "grey")
canevas_droite.pack(padx = 8, pady = 8)

btn_fichier = tk.Button(panneau_boutons, text="Ouvrir")
btn_fichier.pack(fill = tk.X)

def ouvrir_fichier(event):                    
    """ Gestionnaire d'événements
    Ouvre un sélecteur de fichier.
    L'utilisateur choisit une image : elle devient l'image de gauche de l'interface.    """
    global image_originale                   # Variable globale pour stocker l'image PIL sur laquelle nous pourrons travailler
    global image_originale_affichage         # Variable globale pour stocker l'image Tkinter que nous pourrons afficher dans un canvas
    filename = filedialog.askopenfilename()  # Recupère l'adresse du fichier (format str)
    with Image.open(filename) as img:        # Ouvre le fichier Image 
        image_originale = img.copy()                                 # Fait une copie du fichier image pour le garder (et travailler dessus)
        image_originale_affichage = ImageTk.PhotoImage(image = img)  # Transfomer le fichier Image en une image Tkinter pour l'affiher
    canevas_gauche.create_image(0, 0, image = image_originale_affichage, anchor = tk.NW)  # Affiche l'image dans la partie gauche
    
btn_fichier.bind("<Button-1>", ouvrir_fichier) # bouton pour déclencher l'événement

fenetre.mainloop()

### Manipulation d'image
On peut accéder rapidement aux pixels des images avec la classe PixelAccess de PIL (utile pour les filtres). 
On décide d’écrire des filtres qui prennent une image en paramètre (ce sera l’image représentée à gauche), puis créent une autre image et la renvoient (c’est 
l’image qui sera affichée à droite).  
La méthode [`load()`](https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.load) permet d'accéder aux pixels de l'image et donc de faire des manipulations sur les conposantes de ces derniers.  
En aucun cas le filtre ne devra modifier l’image d’entrée.  
Voici un exemple de filtre qui montre comment on parcourt une image (double boucle), et comment on supprime, par exemple, la composante rouge :

In [ ]:
def filtre_supprime_rouge(img1):
    """ Entree : PIL.Image
    Sortie : PIL.Image """
    img2 = img1.copy()
    pimg1 = img1.load()
    pimg2 = img2.load()
    for i in range(img2.size[0]):
        for j in range(img2.size[1]):
            (r, v, b) = pimg1[i, j]
            pimg2[i, j] = (0, v, b)
    return img2

Nous placerons les filtres dans un fichier à part : `filtres.py`.  
Il ne reste plus qu'à associer le filtre à un gestionnaire d'événement.
Nous en profiterons pour écrire une procédure qui affiche dans l’interface graphique le contenu de l'image transformée (car ce dernier point sera commun à toutes les transformations implémentées, il faut donc le coder à part).

In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import ImageTk, Image
import filtres


fenetre = tk.Tk()
fenetre.title("Traitement images")

canevas_gauche = tk.Canvas(fenetre, width = 256, height = 400, background = "grey")
canevas_gauche.pack(padx = 8, pady = 8, side = tk.LEFT)

panneau_boutons = tk.Frame(fenetre)
panneau_boutons.pack(side = tk.LEFT)

canevas_droite = tk.Canvas(fenetre, width = 256, height = 400, background = "grey")
canevas_droite.pack(padx = 8, pady = 8)

btn_fichier = tk.Button(panneau_boutons, text = "Ouvrir")
btn_fichier.pack(fill = tk.X)
btn_suppr_rouge = tk.Button(panneau_boutons, text = "Supprimer Rouge")
btn_suppr_rouge.pack(fill = tk.X)

def ouvrir_fichier(event):
    """ Gestionnaire d'événements
    Ouvre un sélecteur de fichier.
    L'utilisateur choisit une image : elle devient l'image de gauche de l'interface.    """
    global image_originale
    global image_originale_affichage
    filename = filedialog.askopenfilename()
    with Image.open(filename) as img:
        image_originale = img.copy()
        image_originale_affichage = ImageTk.PhotoImage(image = img)
    canevas_gauche.create_image(0, 0, image = image_originale_affichage, anchor = tk.NW)
    
def mise_a_jour_images():
    """ Cette procédure doit être appelée après toute modification de image_transforme. """
    global image_transforme_affichage
    image_transforme_affichage = ImageTk.PhotoImage(image = image_transforme)
    canevas_droite.create_image(0, 0, image = image_transforme_affichage, anchor = tk.NW)

def applique_filtre_supprime_rouge(event):
    """
    Procédure appliquant le filtre à l'image orginale
    """
    global image_transforme
    image_transforme = filtres.filtre_supprime_rouge(image_originale)
    mise_a_jour_images()
        
btn_fichier.bind("<Button-1>", ouvrir_fichier)
btn_suppr_rouge.bind("<Button-1>", applique_filtre_supprime_rouge)

fenetre.mainloop()

# Structure du programme
Tous les éléments précédents pris en compte, nous proposons la structure de code suivante, répartie dans deux fichiers :
* `filtres.py` qui contient les fonctions associées à chaque filtre réalisé.
* `interface_filtres.py` qui contient l’interface graphique et fera le lien avec les filtres du module filtres.py.

Les fichiers sont téléchargeables ici :
* [`filtres.py`](Fichiers/filtres.py)
* [`interface_filtres.py`](Fichiers/interface_filtres.py)
Ainsi que quelques images :
* [`oiseau.png`](Fichiers/oiseau.png)
* [`oeuvre.jpg`](Fichiers/oeuvre.jpg)

# Filtres à implémenter
Image originale : 

<div style="text-align: center">
<img src="Images/oiseau.png" alt="Oiseau">
</div>

## Inverse vidéo
L’inverse vidéo d’une couleur de composantes `(r, v, b)` est la couleur de composantes `(255 – r, 255 – v, 255 – b)`.  
Lorsqu’on réalise un inverse vidéo, le blanc devient noir, le bleu devient jaune, etc.

<div style="text-align: center">
<img src="Images/oiseau_inversion.png" alt="Oiseau">
</div>

## Passage en niveau de gris
Une couleur de composantes `(r, v, b)` est du gris si ses trois composantes sont égales.  
Pour transformer une couleur en niveau de gris, il faut donc trouver une valeur unique, entre 0 et 255 (qu’on placera dans chacun des 3 composante), qui 
reflète la luminosité de la couleur de départ.  
La moyenne des 3 composantes est une première approche.  
Quelques recherches permettent de voir que, l’œil étant plus sensible au vert qu’aux autres couleurs, cette « moyenne » est généralement pondérée.

<div style="text-align: center">
<img src="Images/oiseau_gris.png" alt="Oiseau">
</div>